## Classification
## Example: Predict survival on Titanic

In [10]:
import pandas as pd
import numpy as np

## Working with data

In [64]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [65]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [66]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [67]:
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [68]:
full_data=[train, test]

#### Pclass

In [69]:
# There is no missing value on this feature and already a numerical value.
# let's check it's impact on our train set.

In [70]:
train.groupby('Pclass').size()

Pclass
1    216
2    184
3    491
dtype: int64

In [71]:
train[['Pclass', 'Survived']].groupby('Pclass').Survived.mean()

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

#### Sex

In [72]:
train.groupby('Sex').size()

Sex
female    314
male      577
dtype: int64

In [73]:
train[['Sex', 'Survived']].groupby('Sex', as_index = False).mean()

,Sex,Survived
0,female,0.742038
1,male,0.188908


#### SibSp and Parch

In [74]:
# With the number of siblings/spouse and the number of children/parents 
# we can create new feature called Family Size

In [75]:
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

In [76]:
print(full_data[0].columns)
print(full_data[1].columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'FamilySize'],
      dtype='object')
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'FamilySize'],
      dtype='object')


In [77]:
train[['FamilySize', 'Survived']].groupby('FamilySize', as_index=False).mean()

,FamilySize,Survived
0,1,0.303538
1,2,0.552795
2,3,0.578431
3,4,0.724138
4,5,0.200000
5,6,0.136364
6,7,0.333333
7,8,0.000000
8,11,0.000000


#### Embarked

In [78]:
train[train['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,1
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,1


In [79]:
train[(train['Fare']>79) & (train['Fare']<81) & (train['Pclass']==1)].groupby('Embarked').size()

Embarked
C    4
S    3
dtype: int64

In [80]:
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('C')

In [81]:
train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean()

,Embarked,Survived
0,C,0.558824
1,Q,0.389610
2,S,0.336957


#### Fare

In [82]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

In [83]:
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

In [84]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,CategoricalFare
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,"(-0.001, 7.91]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,"(31.0, 512.329]"
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,"(7.91, 14.454]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,"(31.0, 512.329]"
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,"(7.91, 14.454]"


In [85]:
train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare'], as_index=False).mean()

,CategoricalFare,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


####  Age

In [86]:
# We have plenty of missing values in this feature. 
# Generate random numbers between (mean - std) and (mean + std). 
# Then we categorize age into 5 range.

In [87]:
np.random.seed(0)
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset.loc[dataset['Age'].isnull(), 'Age'] = age_null_random_list 
  
train['CategoricalAge'] = pd.cut(train['Age'], 5)

print (train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge'], as_index=False).mean())

     CategoricalAge  Survived
0    (0.34, 16.336]  0.504274
1  (16.336, 32.252]  0.355056
2  (32.252, 48.168]  0.377510
3  (48.168, 64.084]  0.434783
4    (64.084, 80.0]  0.090909


### Data preprocessing

In [88]:
np.unique(full_data[0]['Sex'], return_counts = True)

(array(['female', 'male'], dtype=object), array([314, 577], dtype=int64))

In [89]:
  for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} )

In [90]:
np.unique(full_data[0]['Embarked'], return_counts = True)

(array(['C', 'Q', 'S'], dtype=object), array([170,  77, 644], dtype=int64))

In [91]:
for dataset in full_data:
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'C': 0, 'Q': 1, 'S': 2} ).astype(int)

In [92]:
for dataset in full_data:   
    # Mapping FamilySize
    dataset.loc[ dataset['FamilySize'] == 1, 'FamilySize'] = 0
    dataset.loc[(dataset['FamilySize'] > 1) & (dataset['FamilySize'] <= 4), 'FamilySize'] = 1
    dataset.loc[dataset['FamilySize'] > 4, 'FamilySize'] = 2 

In [93]:
for dataset in full_data:   
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 64), 'Age'] = 1
    dataset.loc[dataset['Age'] > 64, 'Age'] = 2 

In [94]:
factors_train = train[['Age', 'FamilySize', 'Pclass', 'Embarked']]
factors_test = test[['Age', 'FamilySize', 'Pclass', 'Embarked']]

In [95]:
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder(dtype = 'int32')
enc.fit(factors_train)

OneHotEncoder(categorical_features='all', dtype='int32',
       handle_unknown='error', n_values='auto', sparse=True)

In [96]:
tfactors_train = enc.transform(factors_train).toarray()

In [98]:
tfactors_test = enc.transform(factors_test).toarray()

In [100]:
train_tfact = pd.DataFrame(tfactors_train, columns = ('Age_young','Age_med','Age_old','Fam_small','Fam_med','Fam_large','Pclass_1','Pclass_2','Pclass_3','Emb_C','Emb_Q','Emb_S'))

In [101]:
train_tfact.head()

,Age_young,Age_med,Age_old,Fam_small,Fam_med,Fam_large,Pclass_1,Pclass_2,Pclass_3,Emb_C,Emb_Q,Emb_S
0,0,1,0,0,1,0,0,0,1,0,0,1
1,0,1,0,0,1,0,1,0,0,1,0,0
2,0,1,0,1,0,0,0,0,1,0,0,1
3,0,1,0,0,1,0,1,0,0,0,0,1
4,0,1,0,1,0,0,0,0,1,0,0,1


In [102]:
test_tfact = pd.DataFrame(tfactors_test, columns = ('Age_young','Age_med','Age_old','Fam_small','Fam_med','Fam_large','Pclass_1','Pclass_2','Pclass_3','Emb_C','Emb_Q','Emb_S'))

In [103]:
test_tfact.head()

,Age_young,Age_med,Age_old,Fam_small,Fam_med,Fam_large,Pclass_1,Pclass_2,Pclass_3,Emb_C,Emb_Q,Emb_S
0,0,1,0,1,0,0,0,0,1,0,1,0
1,0,1,0,0,1,0,0,0,1,0,0,1
2,0,1,0,1,0,0,0,1,0,0,1,0
3,0,1,0,1,0,0,0,0,1,0,0,1
4,0,1,0,0,1,0,0,0,1,0,0,1


### Feature Selection

In [104]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Age', 'CategoricalAge', 'CategoricalFare', 'FamilySize', 'Pclass','Embarked']
train = train.drop(drop_elements, axis = 1)
test = test.drop(drop_elements, axis = 1)

In [106]:
train = pd.concat([train,train_tfact], axis=1)
test = pd.concat([test,test_tfact], axis=1)

In [107]:
train.columns

Index(['Survived', 'Sex', 'SibSp', 'Parch', 'Fare', 'Age_young', 'Age_med',
       'Age_old', 'Fam_small', 'Fam_med', 'Fam_large', 'Pclass_1', 'Pclass_2',
       'Pclass_3', 'Emb_C', 'Emb_Q', 'Emb_S'],
      dtype='object')

In [108]:
trainv = train.values

In [109]:
trainv.shape

(891, 17)

In [110]:
type(trainv)

numpy.ndarray

In [111]:
X = trainv[0:, 1:]
y = trainv[0:, 0]